In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


In [ ]:
# Import our input datasets
credit_url = 'https://raw.githubusercontent.com/cindyd97/Project_4_Group_4/main/Resources/Credit_data.csv'
credit_df = pd.read_csv(credit_url)
credit_df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.84,3037.986667,2,...,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x1721,CUS_0x5b48,December,McBrideq,32,706-95-0150,Lawyer,12600.445,782.037083,5,...,11.0,Standard,569.8,30.542582,13 Years and 2 Months,NM,26.060348,19.600997331776377,High_spent_Medium_value_payments,282.542363472351
96,0x172a,CUS_0xa5f9,September,Rickt,34,189-09-5267,Mechanic,57983.12,4720.926667,9,...,17.0,Bad,3532.83,36.791421,12 Years and 10 Months,Yes,200.432613,172.3887516003397,High_spent_Small_value_payments,359.27130167162017
97,0x172b,CUS_0xa5f9,October,Rickt,35,189-09-5267,Mechanic,57983.12,4720.926667,9,...,17.0,_,3532.83,26.583209,12 Years and 11 Months,Yes,200.432613,527.1452187488069,Low_spent_Small_value_payments,34.51483452315301
98,0x172c,CUS_0xa5f9,November,Rickt,35,189-09-5267,Mechanic,57983.12,NaN,9,...,17.0,Bad,3532.83,35.316385,13 Years and 0 Months,Yes,200.432613,191.04338258925824,High_spent_Medium_value_payments,330.6166706827016


In [ ]:
# Check the data types for each column
credit_df.dtypes

,0
ID,object
Customer_ID,object
Month,object
Name,object
Age,object
SSN,object
Occupation,object
Annual_Income,object
Monthly_Inhand_Salary,float64
Num_Bank_Accounts,int64


In [ ]:
# Check the null values in the data set
credit_df.isnull().sum()

In [ ]:
# Drop unnecessary columns (columns that are not needed for our features)
credit_df = credit_df.drop(['ID','Customer_ID','Name','SSN'], axis=1)

In [ ]:
# Convert columns that are meant to be numeric to an integer
columns_to_convert = ['Age','Annual_Income','Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit','Num_Credit_Inquiries','Outstanding_Debt',
                      'Total_EMI_per_month','Amount_invested_monthly','Monthly_Balance']
credit_df[columns_to_convert] = credit_df[columns_to_convert].apply(pd.to_numeric, errors='coerce').astype('float64')

In [ ]:
# Create a function that would convert the strings in 'Credit_History_Age' column to a float. (This column is a string that describes the amount of years and months
# the customer's credit age is)
def convert_to_float(years_months):
    if pd.isna(years_months):
        return 0.0
    parts = years_months.split()
    years = int(parts[0])
    months = int(parts[3])
    return round(years + (months / 12), 2)
# Apply function to convert
credit_df['Credit_History_Age'] = credit_df['Credit_History_Age'].apply(convert_to_float)

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,September,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2022.0,Good,809.98,35.030402,22.75,No,49.574949,236.642682,Low_spent_Small_value_payments,186.266702
1,October,24.0,Scientist,19114.12,1824.843333,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,4.0,Good,809.98,33.053114,22.83,No,49.574949,21.465380,High_spent_Medium_value_payments,361.444004
2,November,24.0,Scientist,19114.12,1824.843333,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,4.0,Good,809.98,33.811894,0.00,No,49.574949,148.233938,Low_spent_Medium_value_payments,264.675446
4,September,28.0,_______,34847.84,3037.986667,2,4,6,1.0,Credit-Builder Loan,...,5.0,Good,605.03,25.926822,27.25,No,18.816215,39.684018,High_spent_Large_value_payments,485.298434
5,October,28.0,Teacher,34847.84,3037.986667,2,4,6,1.0,Credit-Builder Loan,...,5.0,Good,605.03,30.116600,27.33,No,18.816215,251.627369,Low_spent_Large_value_payments,303.355083


In [ ]:
# Logically, the null values in the 'Age' column should not be changed to 0. Remove the rows with null values in this column
credit_df = credit_df.dropna(subset=['Age'])

In [ ]:
# Check the null values in the data set
credit_df.isnull().sum()

,0
Month,0
Age,0
Occupation,0
Annual_Income,3334
Monthly_Inhand_Salary,7122
Num_Bank_Accounts,0
Num_Credit_Card,0
Interest_Rate,0
Num_of_Loan,2312
Type_of_Loan,5416


In [ ]:
# For all null values in 'Type_of_Loan column', update the string to say 'Not Specified'
credit_df['Type_of_Loan'] = credit_df['Type_of_Loan'].fillna('Not Specified')

In [ ]:
# Check the null values in the data set
credit_df.isnull().sum()

,0
Month,0
Age,0
Occupation,0
Annual_Income,3334
Monthly_Inhand_Salary,7122
Num_Bank_Accounts,0
Num_Credit_Card,0
Interest_Rate,0
Num_of_Loan,2312
Type_of_Loan,0


In [ ]:
# The remaining columns with null values may be updated with the number 0
columns_to_fill = ['Annual_Income', 'Monthly_Inhand_Salary', 'Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit','Num_Credit_Inquiries','Outstanding_Debt','Amount_invested_monthly','Monthly_Balance']
credit_df[columns_to_fill] = credit_df[columns_to_fill].fillna(0)

In [ ]:
# Check the null values in the data set
credit_df.isnull().sum()

,0
Month,0
Age,0
Occupation,0
Annual_Income,0
Monthly_Inhand_Salary,0
Num_Bank_Accounts,0
Num_Credit_Card,0
Interest_Rate,0
Num_of_Loan,0
Type_of_Loan,0


In [ ]:
# Download the cleaned data set
credit_df.to_csv('clean_credit_data.csv', index=False)
from google.colab import files
files.download("clean_credit_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>